In [1]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import re

from sklearn.preprocessing import LabelEncoder

data = pd.read_csv('Sentiment.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]

data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

X = pad_sequences(X)

embed_dim = 128
lstm_out = 196
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
# print(model.summary())

labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2)
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model.metrics_names)


<>:18: SyntaxWarning: invalid escape sequence '\s'
<>:18: SyntaxWarning: invalid escape sequence '\s'
C:\Users\ldwen\AppData\Local\Temp\ipykernel_23308\460620488.py:18: SyntaxWarning: invalid escape sequence '\s'
  data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))
C:\Users\ldwen\AppData\Local\Temp\ipykernel_23308\460620488.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row[0] = row[0].replace('rt', ' ')
C:\Users\ldwen\AppData\Local\Temp\ipykernel_23308\460620488.py:21: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  row[0] = row[0].replace('rt', ' ')
c:\Users\ldwen\AppData\Local\

291/291 - 7s - 25ms/step - accuracy: 0.6413 - loss: 0.8292
144/144 - 1s - 8ms/step - accuracy: 0.6656 - loss: 0.7672
0.7671765685081482
0.6655744910240173
['loss', 'compile_metrics']


### 1

In [2]:
model.save("sentiment_analysis.keras")

In [3]:
from tensorflow.keras.models import load_model
model = load_model("sentiment_analysis.keras")

text = 'I really like this class, and I believe that I can get a good grade.'.lower()
sequence = tokenizer.texts_to_sequences([text])
padded_sequence = pad_sequences(sequence)

model.predict(padded_sequence)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step


array([[0.5272558 , 0.1306929 , 0.34205133]], dtype=float32)

### 2

In [25]:
def createmodel(optimizer='adam'):
    model = Sequential()
    model.add(Embedding(max_fatures, 128))
    model.add(LSTM(lstm_out, dropout=.2, recurrent_dropout=.2))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [ ]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

model = KerasClassifier(model=createmodel, epochs=2, batch_size=32)

parameters = {
    'optimizer': ['adam', 'rmsprop'],
    'batch size': [32,64]
}

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=parameters)
grid_result = grid.fit(X_train, Y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))